## Import

In [1]:
import h2o
from h2o.automl import H2OAutoML
import numpy as np
from sklearn.model_selection import train_test_split
import os

## Start

In [ ]:
h2o.init(ip = "localhost",
port = 54321,
start_h2o = True,
max_mem_size="20G",
nthreads = -1)

## Specify grid name

In [3]:
net_name='net_95_v1'

## Data Loading

In [5]:
alt_x = np.load(f'./data/{net_name}/measured_data_x_alt.npy')
alt_y = np.load(f'./data/{net_name}/data_y_alt.npy')
data_x = alt_x
data_y = alt_y

split_train = int(0.8 * data_x.shape[0])
train_x = data_x[:split_train, :]
train_y = data_y[:split_train, :]

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, shuffle=True, random_state=42)

train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.3, shuffle=True, random_state=42)

num_input = 206
num_output = 95

in_columns = [str(i) for i in range(num_input)]
out_columns = [str(i) for i in range(num_input, num_input + num_output)]

## Train and save models (one model per output node)

In [ ]:
time_limit=60

In [ ]:
%%time

for i in range(num_output):
    if not os.path.exists(f'./h2o_models_{time_limit}_v1'):
        os.makedirs(f'./h2o_models_{time_limit}_v1')

for i in range(num_output):
    print(f'Training for output #{i}')
    train = np.hstack((train_x, train_y[:, i].reshape(-1, 1)))
    columns_names = in_columns + [out_columns[i]]
    train = h2o.H2OFrame(train, column_names=columns_names)
    
    x = in_columns
    y = out_columns[i]
    aml = H2OAutoML(max_models=2, seed=1, max_runtime_secs_per_model=time_limit)
    aml.train(x=x, y=y, training_frame=train)
    
    model_path = f'./h2o_models_{time_limit}_v1/model_{i}'
    aml = h2o.save_model(model=aml.leader, path=model_path, force=True)

## Load models (when necessary)

In [ ]:
models = []
for i in range(num_output):
    model_path = f'./h2o_models_{time_limit}_v1/model_{i}/'
    files = os.listdir(model_path)
    model_filename = [f for f in files if os.path.isfile(os.path.join(model_path, f))][0]
    
    aml = h2o.load_model(f'{model_path}/{model_filename}')
    models.append(aml)
    print(f'Model {i} loaded')

## Collect metric for validation data

In [ ]:
from net95.scenarios2 import get_data_by_scenario_and_case

std_results = []
for scenario in range(1, 2):
    for case in range(1, 2):
        print(f'SCENARIO {scenario}, CASE {case} VALIDATION')
        s1_c1_data = get_data_by_scenario_and_case(scenario, case, net_name='net95v1')
        x = s1_c1_data[0]
        x_hat = s1_c1_data[1]
        y_all = s1_c1_data[2]
        y_hat_all = s1_c1_data[3]
        
        estim = []
        for i in range(num_output):
            columns_names = in_columns + [out_columns[i]]
            x = in_columns
            y = out_columns[i]
            
            aml = models[i]
            test_x = x_hat
            test_y = np.asarray(y_all[0][i]).reshape(-1, 1)
            test = h2o.H2OFrame(np.hstack((test_x, test_y)), column_names=columns_names)
            
            try:
                preds = aml.leader.predict(test)
            except:
                preds = aml.predict(test)
            estim.append(preds['predict'].as_data_frame().iloc[0][0])
            
        pred = np.asarray(estim)
        #report_preds_on_validation_files(pred, 9, 'h2o', scenario, case=case)
        if case == 1:
            std_results.append(f'std: {np.sqrt(np.mean(np.square(y_all - pred)))}')
print(std_results)
print(preds)
print(y_all)


## Generate local explanations with shap
In this case local explanations are generated for the output index #30 and are applied to explain the models' output when the input correspond to scenario 1 case 1.

In [ ]:
import shap
import pandas as pd
from net95.scenarios2 import get_data_by_scenario_and_case

import matplotlib.pyplot as plt

np.random.seed(42)
scenario=1
case=1
node_index = 30
predictor = models[node_index] # I want to explain state estimation for node at index 30
columns_names = in_columns + [out_columns[node_index]]

def wrapped_model(x):
    x = h2o.H2OFrame(x)
    x.col_names = columns_names[:-1]
    preds = predictor.predict(x).as_data_frame().to_numpy()[:, 0]
    return preds

s1_c1_data = get_data_by_scenario_and_case(scenario, case, net_name='net95v1')
x = s1_c1_data[0]
x_hat = s1_c1_data[1]
y_all = s1_c1_data[2]
y_hat_all = s1_c1_data[3]
to_be_explained = x

random_indices = np.random.choice(test_x.shape[0], size=100, replace=False)
explainer = shap.KernelExplainer(wrapped_model, train_x[random_indices])
shap_values = explainer.shap_values(to_be_explained)
relevance = abs(shap_values.ravel())

x_positions = np.arange(len(relevance))
plt.figure(figsize=(15,6))
plt.bar(x_positions, relevance, color='green')
plt.xlabel('Model input index')
plt.ylabel('Contributions')
plt.title(f'SHAP Values for H2O - output index {node_index}')
#plt.xticks(x_positions, ['A', 'B', 'C', 'D', 'E'])
print(sorted([(i, j) for i,j in enumerate(relevance)], key=lambda t: -t[1])[:10])
plt.show()


fig, ax = plt.subplots()
labels = ['p_mw', 'q_mvar', 'vm_pu', 'p_mw_lines', 'q__mvar_lines']
aggregate_data = [sum(relevance[:94])/94., sum(relevance[94:94+94])/94., sum(relevance[94+94:94+94+4])/4., sum(relevance[94+94+4:94+94+4+7])/7., sum(relevance[94+94+4+7:])/7.]
ax.pie(aggregate_data, labels=labels, autopct='%1.1f%%')


fig, ax = plt.subplots()
labels = ['vm_pu', 'other']
aggregate_data = [sum(relevance[94+94:94+94+4])/4., sum(relevance[:94])/94. + sum(relevance[94:94+94])/94. + sum(relevance[94+94+4:94+94+4+7])/7. + sum(relevance[94+94+4+7:])/7.]
ax.pie(aggregate_data, labels=labels, autopct='%1.1f%%')


fig, ax = plt.subplots()
labels = ['p_mw', 'q_mvar', 'p_mw_lines', 'q__mvar_lines']
aggregate_data = [sum(relevance[:94])/94., sum(relevance[94:94+94])/94., sum(relevance[94+94+4:94+94+4+7])/7., sum(relevance[94+94+4+7:])/7.]
ax.pie(aggregate_data, labels=labels, autopct='%1.1f%%')




'''
norm_relevance = ((relevance-abs(relevance)) / (max(relevance) - min(relevance)))

print(relevance)
plt.imshow(norm_relevance.reshape((53, 1)))
plt.colorbar()
'''